# LSTM network for language modeling
Here we'll define our own lstm forward pass from scracth, we'll let on pytorch's graph do the backward pass for us. We feed the network one char at a time and let it predict the next char for us. This implementation is inspired by Andrej Karpathy's charRNN and minGPT.

<img src='images/General-scheme-of-an-Long-Short-Term-Memory-neural-networks-LSTM-for-L-p-The.ppm' width=600>
<a href=https://www.researchgate.net/figure/General-scheme-of-an-Long-Short-Term-Memory-neural-networks-LSTM-for-L-p-The_fig1_339120709>Image from</a>

The six equations that make up LSTM
```python
# concat is input, bxs are bias, Wxs are weights
        f = torch.sigmoid(self.Wf @ concat + self.bf) # forget gate
        u = torch.sigmoid(self.Wu @ concat + self.bu) # update gate
        cc = torch.tanh(self.Wcc @ concat + self.bcc) # candidate
        o = torch.sigmoid(self.Wo @ concat + self.bo) # output gate

        c = f*c_prev + u*cc # cell state
        a = o*torch.tanh(c) # hidden state

```

In [1]:
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

%load_ext autoreload
%autoreload 2
%matplotlib inline
from utils.utils import pickle, set_seed
set_seed(12)

In [2]:
from torch.utils.data import Dataset
import torch
import numpy as np

class CharDataset(Dataset):
    def __init__(self, data, sequence_len):
        # data in the type of pairs of sentence
        
        chars = ['<pad>']+sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.ch2i = { ch:i for i,ch in enumerate(chars) }
        self.i2ch = { i:ch for i,ch in enumerate(chars) }
        self.sequence_len = sequence_len
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.sequence_len
    
    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.sequence_len + 1]
        # encode every character to an integer
        dix = [self.ch2i[s] for s in chunk]

        x = torch.tensor(self.vectorization(dix[:-1]), dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        
        
        return x,y
                                                                                                                                       
    def padding(self, string):
        if len(string)<self.sequence_len:
            string =  string + [0]*(self.sequence_len - len(string))
        else:
            string = string[:self.sequence_len]
                   
        return string
    
    def vectorization(self, idx):
        out = np.zeros((self.sequence_len, self.vocab_size))
        for i, index in enumerate(idx):
            out[i,index] = 1
        
        return out

In [15]:
text = open('data/Tuyen_Tap_To_Hoai_To_Hoai.txt', 'r', encoding='utf-8').read() # replace text data here
dataset = CharDataset(text, sequence_len=128)

data has 3892297 characters, 200 unique.


# Train network

In [11]:
from model.lstm import LSTM, LSTMConfig
from utils.trainer import Trainer, TrainerConfig

tconfig = TrainerConfig(max_epochs=1, 
                        batch_size=512, 
                        learning_rate=0.01, 
                        grad_norm_clip=1.0, 
                        device='cuda',
                        lr_decay=True, 
                        warmup_tokens=5000, 
                        ckpt_n_print_iter=2000, 
                        ckpt_path='checkpoint/lstm')

mconfig = LSTMConfig(vocab_size = dataset.vocab_size,
                        sequence_len = dataset.sequence_len,
                        hidden_size = 256,
                        batch_size = tconfig.batch_size,
                        device = tconfig.device)

In [12]:
lstm = LSTM(mconfig)
trainer = Trainer(lstm, dataset, tconfig, test_dataset=None, collate=None)

09/25/2021 05:18:22 - INFO - model.lstm -   number of parameters: 5.193680e+05


In [13]:
lstm.load_state_dict(pickle(tconfig.ckpt_path)) # load

<All keys matched successfully>

In [14]:
trainer.train()

epoch: 1 | train loss: 0.52646  | lr: 1.024000e-03:   0%|                                     | 0/7601 [00:00<?, ?it/s]

 tht   nhittnhi n nt it nnhninhttttihhn n i ihinh i ihh n itn thhhn th  nhi htn hi nii  h  ii hh inn  itintt tti thh  nithnihi hhn   tni hihhii      th it t h iinti   thhhhh t  iii it  h t  t tt h ih iti nhhh itt nhti hhit  thiinh nht tthh  hht hn nhhttnthi h ih ihni inh nnin tth ntt th h tnh hti n nh ihnh  t hiit ihhht  iit ii nhtn   tt hh  t   tih n  n  t itnh ihtn nninihnhihnnh i h  tnhn  nh n  h innni hiintiiinh t ihn  ihhnn hh ih n    nt itiinnnnhi hin i   iit hinin ttinn ninn nhhn nnhiniitnhh nhhhhhhnhht hhthn nth nintt hh ni hhnthhhi   h ih niinnhihii  int ihinttni h  t  i hhihhtthhnin   tthn t nhn hnii hnin n h thh tht hiit ntt  t  nhih  it th  ntn n nn tthii ntnnhnt th hhhnnhi hih iht nihhn t h n n ti iii  i n n tn  htn nhntnitiinh ni  hh  nnhh   nthnht  iin i tnhnhh t ti i  nt  t h  in  htn   ith nhn ihnhiin  h  h itniihhnhhhit n innii  tnhh iihttinn 


epoch: 1 | train loss: 1.48452  | lr: 8.396229e-03:  26%|██████▊                   | 2000/7601 [09:33<26:15,  3.55it/s]

 chiếc Quốc dân thuyết Nhà Trò nghiêm nghị làm nhà máy chân chim gáy trong làng chính phủ Quốc Hoàng Liên hoàng cho tôi người chết cho tôi được đi đâu mà đến chỗ có cái đầu chiếc đồi báo cho những người đi trong tháng ngày đã đi trong các chủ nhà chúng tôi đã thấy cái cánh đồng của nhà tôi. Nhưng chẳng bao lâu, tôi nghĩ thế, như tôi nghĩ ra đường trong làng. Thế mà cũng không có những đứa trẻ trong cái gác cái chuông trong bóng cái bè ngọn núi, chẳng bao lâu chỉ làm giác nghề nghiên chui vào trong làng những câu chuyện cũng chẳng bao giờ làm thế nào cũng là một cái thúng thấy một bà con trên cái chợ chúng tôi thì chẳng khác cái thằng đánh chuyện và thế nào? Thế mà đã nghe tiếng trống đưa đến chỗ nào cũng có những đám nhiều chiếc đá như thế này. Thằng này thì con cá mật thám được chân trời đất trong bụi cây trên trời trắng như những bóng núi vào mặt chân như thế


epoch: 1 | train loss: 1.42906  | lr: 4.595666e-03:  53%|█████████████▋            | 4000/7601 [19:12<17:24,  3.45it/s]

  không phải đội trưởng Cự đã đi đâu về thành phố trên thế giới cho tôi chỉ có một chiếc đòn chạy vào đây. Nhưng cái ngày trước đã thành một chuyện đất và báo cho người chết cho nhà thường trên báo Cứu Quốc vào trong làng cho được đi đâu cũng thế. Nhưng tôi đã chết như một chuyện đấy là cái thằng này thì thào thế nào được cho các cụ trong lớp trong chiến trường tiểu thuyết Thìn ngày thứ bảy như thế chứ chẳng được cái truyện ngắn Thiệp trường Yên Phục Quốc như thế, chẳng bao giờ thấy trên đường cái cho ta nghĩ ngợi thế. Tôi chỉ có một cái thư một mảng đá người ngoài cửa hang, có ngọn đèn ngoài trời đất trong bếp đã lâu. Trời đã trông thấy đi chợ cũng có thể thế nào?

- Các chị trưởng thôn trong truyện ngắn Những ngày trước, tôi thường một người nào thì chết thế nào cũng là cái đuôi người. Nhưng chỉ có một người trong khi thì chẳng có tiền trong thành phố thường 


epoch: 1 | train loss: 1.41047  | lr: 1.056644e-03:  79%|██████████████████▉     | 6000/7601 [4:49:37<07:36,  3.51it/s]

 







Chương Iv (4):





Mục lục


:





Một thôi chuột thế nào thì chúng tôi lại thấy tôi được châu đoàn thế nào đã biết thế nào. Chú thích nghe theo địa phận cho thấy trong thành phố đã thành thói quen chơi cho đến lúc cái thuyền về truyện ngắn thịt gà của người ta vừa khác như mấy nhà cho mình được. Chẳng biết được một tiếng hát chưa?

Chèo Bẻo chồng chạy lên cho nhau thế này. Đến khi đi vào nhà trong trước cái thanh lưỡi, thế mà thôi cho con người như thế. Tôi có người nhìn thấy bàn chân chiếc thùng cái chân chim thì thào như người đi làm gì?

- Thế là được cho chú chuyện trên đầu trong những trận mưa rào rào ràn đi trước đấy. Những con chim đi được cái chân cánh đồng làng trong những những con chim già đã làm thế nào cũng chẳng được cái thúng cho người ta ngoài đồng. Nhưng đã thấy cái chợ đương cầu bò của tôi ngồi trên đầu cánh đồng cho chúng tôi. Tôi


epoch: 1 | train loss: 1.40585  | lr: 1.000000e-03: 100%|████████████████████████| 7601/7601 [4:57:12<00:00,  2.35s/it]


# GPT Character Transformer model 
<img src='images/The-scaled-dot-product-attention-and-multi-head-self-attention.png' width=600>

We use build a gpt like model that uses Transformer architecture. The main component is multihead self-attention which is implemented in python like so. The causal mask is to prevent network from looking ahead.
```python
        dk = embed_dim//n_head
        
        q = self.query(q).view(batch_size, sequence_len, n_head, dk).transpose(1,2) # B,nh,T,dk
        k = self.key(k).view(batch_size, sequence_len, n_head, dk).transpose(1,2)
        v = self.value(v).view(batch_size, sequence_len, n_head, dk).transpose(1,2)
        # scale q and k
        att = (q @ k.transpose(-2,-1))/(dk**0.5) # B,nh,T,dk x B,nh,dk,T -> B,nh,T,T
        if causal is not None:
            att = att.masked_fill(self.causal_mask == 0, float('-1e10'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v # B,nh,T,T x B,Nh,T,dk -> B,nh,T,dk
        # swap n_head back to last dimension then re-assemble side by side to embed_dim
        y = y.transpose(1,2).contiguous().view(batch_size, sequence_len, n_head*dk) 
        # output projection
        y = self.resid_drop(self.proj(y)) # output is batch_size, sequence_len, embed_dim
```

In [16]:
from model.transformer import Transformer, TansformerConfig
from utils.trainer import Trainer, TrainerConfig

tconfig = TrainerConfig(max_epochs=1, 
                        batch_size=64, 
                        learning_rate=6e-4, 
                        grad_norm_clip=1.0, 
                        device='cuda',
                        lr_decay=True, 
                        warmup_tokens=5000, 
                        ckpt_n_print_iter=2000, 
                        ckpt_path='checkpoint/transformer')

mconfig = TansformerConfig(vocab_size = dataset.vocab_size,
                      sequence_len = dataset.sequence_len,
                      embed_dim = 256,
                      n_block=4,
                      n_head=8,
                      causal=True,
                      device = tconfig.device)

In [17]:
gpt = Transformer(mconfig)
trainer = Trainer(gpt, dataset, tconfig, test_dataset=None, collate=None)

09/25/2021 10:19:18 - INFO - model.transformer -   number of parameters: 3.291904e+06


In [18]:
gpt.load_state_dict(pickle(tconfig.ckpt_path))

<All keys matched successfully>

In [13]:
trainer.train()

epoch: 1 | train loss: 0.53366  | lr: 8.160000e-06:   0%|                                    | 0/57237 [00:00<?, ?it/s]

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> [[ŒỪZ11ZZ[[ZỀŒZỪỀỀ[ZŒŒŒỪZZŒỪỪẳỪẳặMŒỪọ1ỪZ![ZZ11Z!ŒZỪỪMŒŒặ!Z[Z1ẳặMZZ![111ỪọỪM1ŒặŒMỪỪỪẳặZZ1ŒM1<pad>Ừẳ!Z<pad>MÒ<pad>M<pad>MÒỪỪ<pad>MÒuốu<pad>ú<pad>u<pad><pad><pad>ú<pad>ầ PÝ ni i  ni g    i  i n     h     i h  h   i   i    Ơ            h h     h              i  n   h   h g      i          q       h g    h i      g    n   h  n     i           h       n  n     h i  n 

epoch: 1 | train loss: 1.65169  | lr: 5.983184e-04:   3%|▊                      | 2000/57237 [05:12<2:21:42,  6.50it/s]

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> chươi trên trên mặt thành ngang ngày đường có chiếc được chiếc đấu chủ được nghĩa cho tôi được chúng tôi đi chơi thì tôi làm chẳng thấy tôi chẳng biết thế mà chúng tôi không có như con chim được một người ta đã biết thế. Nhưng mà chỉ có thể thấy một cái làm thế, tôi cũng đã biết cho tôi cho chúng tôi thấy tôi chỉ có thể chuyện nghĩ tôi. Một chuyện tranh nghề làm

epoch: 1 | train loss: 1.52809  | lr: 5.930387e-04:   7%|█▌                     | 4000/57237 [10:24<2:17:09,  6.47it/s]

...

epoch: 1 | train loss: 1.20142  | lr: 6.000000e-05:  94%|████████████████████▊ | 54002/57237 [2:15:34<41:50,  1.29it/s]

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> làm! Tai nghe tiếng một tiếng. Những chuồng gà đã thanh thảnh thảnh thơi rồi lại như thế.

Chẳng bao lâu, chỉ có một người đàn bà thì chạy theo. Trên chiếc thuyền canh cuộn tròn như người đàn bà liềm thấy mình thì chết đói lại chết. Người nhà bàn tán thì chết rồi. Người ta bảo thế. Người ta nói người ta đấy, nhưng tôi cũng chưa được biết thế nào. Thế là những cá

epoch: 1 | train loss: 1.20292  | lr: 6.000000e-05:  98%|█████████████████████▌| 56002/57237 [2:20:43<16:17,  1.26it/s]

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> m o ooai như một chút. Tôi chỉ nhìn thấy thầy giáo tôi được một nghe tiếng giọng cười nói nhỏ mà thôi. Thầy giáo tôi chỉ nhớ mười năm trước, thầy cho chúng tôi biết cái chăn dạ. Thầy giáo tôi cũng nhớ ra những ngày thầy giáo ngày xưa cái thư tình cờ tôi chỉ có một thôi. Thầy giáo ngồi xuống đầu gối đầu gối đầu gối. Có thể chết thôi. Đứa nào chết chém ngay được t

epoch: 1 | train loss: 1.20018  | lr: 6.000000e-05: 100%|██████████████████████| 57237/57237 [2:23:54<00:00,  6.63it/s]


## Let's compare the result

In [19]:
sample = "cháu lên ba cháu vô mẫu giáo"
print(lstm.generate_output(sample, dataset, top_k=5, temperature=0.7))

cháu lên ba cháu vô mẫu giáo





Mục lục


Mùi:

- Mày biết đi đâu cũng thày tôi được. Nhưng những tiếng thầy có người đã bắt chước và trong làng, cho đến đâu về làng, người lớn cho chúng mày thế thì thế nào. Mon nhìn đi được. Bác Thậm nói là người ngày ngày đi làm chỗ trong nhà đã bắt được một cái nghiêng thôn thờ các con và tiếng chuồn nhưng lại nhìn thấy tôi lại đến đây này?

- Thế thì đến đứa thang tráng ngoài thành phố. Thông thường. Một tháng chi liệu ngày nào cũng có người cũng đến được một chuyện thật thì cái trường thì chẳng có được điếm thuộc nhà tôi. Các nhân vật cho chúng tôi đã đi đâu. Nhưng trong những cuộc truyện ngắn Tân Dân đã thành phố trong những buồng thì nghi ngờ, chúng tôi chỉ có tiếng thì có tiếng thú vịt được đi chơi. Chỉ nghi ngờ ngang người ta cho chúng tôi.

Năm 1957, chẳng điều thu từ đây làm cho người ta đã trở về đây, không biết thế nào cũng chỉ được thằng 


In [20]:
print(gpt.generate_output(sample, dataset, top_k=5, temperature=0.7))

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>cháu lên ba cháu vô mẫu giáo. Thế là thầy giáo Hoạnh trên mặt trận Bình dân tộc phản động, báo động chí đã biết chưa?

Chẳng may, tôi không biết, nhưng tôi không thể biết chừng. Tôi chỉ làm cho có thế thôi. Có lần anh đương đứng lại trên đầu cầu tay. Tôi nghĩ có cái đầu tôi như chuyện cái gì thế này. Tôi không thể coi như một lúc sau. Tôi đi tìm đến tập cuộc đời có người đọc sống trong bóng tối. Tôi không hiểu tại sao, không thấy đội trưởng Cự lại đứng lại, thì thào về nhà. Đến lúc cả nhà đã đượ

We can see Transformer model retains the long range relation/dependency much more than LSTM, hence produce more meaningful sentences.

**Take away:**
- Transformer works better than LSTM at language modeling for the purpose of text generation
- Transformer generally takes longer to train due to the sheer amount of parameters
- We dont have very good metric to discrimite good and bad text